<a href="https://colab.research.google.com/github/vikpy/basic_visualzation_in_R/blob/master/bentoml_quick_start_guide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting Started with BentoML

[BentoML](http://bentoml.ai) is an open-source framework for machine learning **model serving**, aiming to **bridge the gap between Data Science and DevOps**.

Data Scientists can easily package their models trained with any ML framework using BentoMl and reproduce the model for serving in production. BentoML helps with managing packaged models in the BentoML format, and allows DevOps to deploy them as online API serving endpoints or offline batch inference jobs, on any cloud platform.

This getting started guide demonstrates how to use BentoML to serve a sklearn modeld via a REST API server, and then containerize the model server for production deployment.

![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=guides&ea=bentoml-quick-start-guide&dt=bentoml-quick-start-guide)

BentoML requires python 3.6 or above, install dependencies via `pip`:

In [1]:
# Install PyPI packages required in this guide, including BentoML
!pip install -q --pre bentoml 
!pip install -q 'scikit-learn>=0.23.2' 'pandas>=1.1.1'

     |████████████████████████████████| 3.7MB 7.9MB/s 
     |████████████████████████████████| 133kB 45.4MB/s 
     |████████████████████████████████| 61kB 7.6MB/s 
     |████████████████████████████████| 1.2MB 48.7MB/s 
     |████████████████████████████████| 112kB 51.5MB/s 
     |████████████████████████████████| 92kB 8.9MB/s 
     |████████████████████████████████| 81kB 8.3MB/s 
     |████████████████████████████████| 163kB 46.7MB/s 
     |████████████████████████████████| 61kB 6.7MB/s 
     |████████████████████████████████| 2.7MB 41.9MB/s 
     |████████████████████████████████| 153kB 42.5MB/s 
     |████████████████████████████████| 133kB 48.1MB/s 
     |████████████████████████████████| 6.7MB 40.9MB/s 
     |████████████████████████████████| 71kB 6.7MB/s 
     |████████████████████████████████| 296kB 44.4MB/s 
     |████████████████████████████████| 153kB 43.7MB/s 
     |████████████████████████████████| 552kB 42.8MB/s 
     |████████████████████████████████| 102kB 9.3MB/s 
    

Before starting, let's prepare a trained model for serving with BentoML. Train a classifier model on the [Iris data set](https://en.wikipedia.org/wiki/Iris_flower_data_set):

In [2]:
import sklearn as sklearn 
sklearn.__version__

'0.23.2'

In [3]:
import pandas as pd 
pd.__version__

'1.1.2'

In [ ]:
!pip --version

pip 20.1 from c:\python37\lib\site-packages\pip (python 3.7)



In [4]:
from sklearn import svm
from sklearn import datasets

# Load training data
iris = datasets.load_iris()
X, y = iris.data, iris.target

# Model Training
clf = svm.SVC(gamma='scale')
clf.fit(X, y)

SVC()

## Create a Prediction Service with BentoML

Model serving with BentoML comes after a model is trained. The first step is creating a
prediction service class, which defines the models required and the inference APIs which
contains the serving logic. Here is a minimal prediction service created for serving
the iris classifier model trained above:

In [5]:
%%writefile iris_classifier.py
import pandas as pd

from bentoml import env, artifacts, api, BentoService
from bentoml.adapters import DataframeInput
from bentoml.frameworks.sklearn import SklearnModelArtifact

@env(infer_pip_packages=True)
@artifacts([SklearnModelArtifact('model')])
class IrisClassifier(BentoService):
    """
    A minimum prediction service exposing a Scikit-learn model
    """

    @api(input=DataframeInput(), batch=True)
    def predict(self, df: pd.DataFrame):
        """
        An inference API named `predict` with Dataframe input adapter, which codifies
        how HTTP requests or CSV files are converted to a pandas Dataframe object as the
        inference API function input
        """
        return self.artifacts.model.predict(df)

Writing iris_classifier.py


This code defines a prediction service that packages a scikit-learn model and provides
an inference API that expects a `pandas.Dataframe` object as its input. BentoML also supports other API input 
data types including `JsonInput`, `ImageInput`, `FileInput` and 
[more](https://docs.bentoml.org/en/latest/api/adapters.html).


In BentoML, **all inference APIs are suppose to accept a list of inputs and return a 
list of results**. In the case of `DataframeInput`, each row of the dataframe is mapping
to one prediction request received from the client. BentoML will convert HTTP JSON 
requests into :code:`pandas.DataFrame` object before passing it to the user-defined 
inference API function.
 
This design allows BentoML to group API requests into small batches while serving online
traffic. Comparing to a regular flask or FastAPI based model server, this can increases
the overall throughput of the API server by 10-100x depending on the workload.

The following code packages the trained model with the prediction service class
`IrisClassifier` defined above, and then saves the IrisClassifier instance to disk 
in the BentoML format for distribution and deployment:

In [6]:
# import the IrisClassifier class defined above
from iris_classifier import IrisClassifier

# Create a iris classifier service instance
iris_classifier_service = IrisClassifier()

# Pack the newly trained model artifact
iris_classifier_service.pack('model', clf)

# Save the prediction service to disk for model serving
saved_path = iris_classifier_service.save()

[2020-10-18 06:09:58,259] WARNING - pip package requirement `bentoml==0.9.2` not found in current python environment


/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


[2020-10-18 06:09:59,814] INFO - BentoService bundle 'IrisClassifier:20201018060959_B823A8' saved to: /root/bentoml/repository/IrisClassifier/20201018060959_B823A8


BentoML stores all packaged model files under the
`~/bentoml/{service_name}/{service_version}` directory by default.
The BentoML file format contains all the code, files, and configs required to 
deploy the model for serving.


## REST API Model Serving



To start a REST API model server with the `IrisClassifier` saved above, use 
the `bentoml serve` command:

In [7]:
!bentoml serve IrisClassifier:latest --run-with-ngrok

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
[2020-10-18 06:10:09,075] INFO - Getting latest version IrisClassifier:20201018060959_B823A8
[2020-10-18 06:10:09,076] INFO - Starting BentoML API server in development mode..
 * Serving Flask app "IrisClassifier" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
[2020-10-18 06:10:15,004] INFO -  * Running on http://9164737b161b.ngrok.io
[2020-10-18 06:10:15,004] INFO -  * Traffic stats available on http://127.0.0.1:4040
127.0.0.1 - - [18/Oct/2020 06:10:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2020 06:10:19] "GET /swagger_static/swagger-ui.css HTTP/1

If you are running this notebook from Google Colab, you can start the dev server with `--run-with-ngrok` option, to gain acccess to the API endpoint via a public endpoint managed by [ngrok](https://ngrok.com/): 

In [ ]:
!bentoml serve IrisClassifier:latest --run-with-ngrok

^C


The `IrisClassifier` model is now served at `localhost:5000`. Use `curl` command to send
a prediction request:

```bash
curl -i \
--header "Content-Type: application/json" \
--request POST \
--data '[[5.1, 3.5, 1.4, 0.2]]' \
localhost:5000/predict
```

Or with `python` and [request library](https://requests.readthedocs.io/):
```python
import requests
response = requests.post("http://127.0.0.1:5000/predict", json=[[5.1, 3.5, 1.4, 0.2]])
print(response.text)
```

Note that BentoML API server automatically converts the Dataframe JSON format into a
`pandas.DataFrame` object before sending it to the user-defined inference API function.

The BentoML API server also provides a simple web UI dashboard.
Go to http://localhost:5000 in the browser and use the Web UI to send
prediction request:

![BentoML API Server Web UI Screenshot](https://raw.githubusercontent.com/bentoml/BentoML/master/guides/quick-start/bento-api-server-web-ui.png)

## Containerize model server with Docker



One common way of distributing this model API server for production deployment, is via
Docker containers. And BentoML provides a convenient way to do that.

Note that `docker` is __note available in Google Colab__. You will need to download and run this notebook locally to try out this containerization with docker feature.

If you already have docker configured, simply run the follow command to product a 
docker container serving the `IrisClassifier` prediction service created above:

In [ ]:
!bentoml containerize IrisClassifier:latest -t iris-classifier

[2020-10-18 11:32:42,907] INFO - Getting latest version IrisClassifier:20201018111338_D2897D
Found Bento: C:\Users\Vikraant Pai/bentoml/repository/IrisClassifier\20201018111338_D2897D
Image version not specified, using version parsed from BentoService: '20201018111338_D2897D'
Building Docker image iris-classifier:20201018111338_D2897D from IrisClassifier:latest 



Traceback (most recent call last):
  File "c:\python37\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\python37\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Python37\Scripts\bentoml.exe\__main__.py", line 7, in <module>
  File "c:\python37\lib\site-packages\click\core.py", line 764, in __call__
    return self.main(*args, **kwargs)
  File "c:\python37\lib\site-packages\click\core.py", line 717, in main
    rv = self.invoke(ctx)
  File "c:\python37\lib\site-packages\click\core.py", line 1137, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "c:\python37\lib\site-packages\click\core.py", line 956, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "c:\python37\lib\site-packages\click\core.py", line 555, in invoke
    return callback(*args, **kwargs)
  File "c:\python37\lib\site-packages\bentoml\cli\click_utils.py", line 138, in wrapper
    return func(*args, **kwargs)
  File "

Start a container with the docker image built in the previous step:

In [ ]:
!docker run -p 5000:5000 iris-classifier:latest --workers=1 --enable-microbatch

'docker' is not recognized as an internal or external command,
operable program or batch file.


This made it possible to deploy BentoML bundled ML models with platforms such as
[Kubeflow](https://www.kubeflow.org/docs/components/serving/bentoml/),
[Knative](https://knative.dev/community/samples/serving/machinelearning-python-bentoml/),
[Kubernetes](https://docs.bentoml.org/en/latest/deployment/kubernetes.html), which
provides advanced model deployment features such as auto-scaling, A/B testing,
scale-to-zero, canary rollout and multi-armed bandit.


## Load saved BentoService

`bentoml.load` is the API for loading a BentoML packaged model in python:

In [ ]:
import bentoml
import pandas as pd

bento_svc = bentoml.load(saved_path)

# Test loaded bentoml service:
bento_svc.predict([X[0]])

[2020-10-18 11:32:45,400] WARNING - Module `iris_classifier` already loaded, using existing imported module.
[2020-10-18 11:32:45,408] WARNING - pip package requirement pandas already exist
[2020-10-18 11:32:45,411] WARNING - pip package requirement scikit-learn already exist


memmap([0])

The BentoML format is pip-installable and can be directly distributed as a
PyPI package for using in python applications:

In [ ]:
!pip install -q {saved_path}

ERROR: Invalid requirement: 'C:\\Users\\Vikraant'
Hint: It looks like a path. File 'C:\Users\Vikraant' does not exist.
You should consider upgrading via the 'c:\python37\python.exe -m pip install --upgrade pip' command.


In [ ]:
# The BentoService class name will become packaged name
import IrisClassifier

installed_svc = IrisClassifier.load()
installed_svc.predict([X[0]])

ModuleNotFoundError: No module named 'IrisClassifier'

This also allow users to upload their BentoService to pypi.org as public python package
or to their organization's private PyPi index to share with other developers.

`cd {saved_path} & python setup.py sdist upload`

*You will have to configure ".pypirc" file before uploading to pypi index.
    You can find more information about distributing python package at:
    https://docs.python.org/3.7/distributing/index.html#distributing-index*


# Launch inference job from CLI

BentoML cli supports loading and running a packaged model from CLI. With the `DataframeInput` adapter, the CLI command supports reading input Dataframe data from CLI argument or local `csv` or `json` files:

In [ ]:
!bentoml run IrisClassifier:latest predict --input='[[5.1, 3.5, 1.4, 0.2]]'

In [ ]:
!bentoml run IrisClassifier:latest predict \
    --input-file="./iris_data.csv"

# Deployment Options

If you are at a small team with limited engineering or DevOps resources, try out automated deployment with BentoML CLI, currently supporting AWS Lambda, AWS SageMaker, and Azure Functions:
  - [AWS Lambda Deployment Guide](https://docs.bentoml.org/en/latest/deployment/aws_lambda.html)
  - [AWS SageMaker Deployment Guide](https://docs.bentoml.org/en/latest/deployment/aws_sagemaker.html)
  - [Azure Functions Deployment Guide](https://docs.bentoml.org/en/latest/deployment/azure_functions.html)

If the cloud platform you are working with is not on the list above, try out these step-by-step guide on manually deploying BentoML packaged model to cloud platforms:
  - [AWS ECS Deployment](https://docs.bentoml.org/en/latest/deployment/aws_ecs.html)
  - [Google Cloud Run Deployment](https://docs.bentoml.org/en/latest/deployment/google_cloud_run.html)
  - [Azure container instance Deployment](https://docs.bentoml.org/en/latest/deployment/azure_container_instance.html)
  - [Heroku Deployment](https://docs.bentoml.org/en/latest/deployment/heroku.html)

Lastly, if you have a DevOps or ML Engineering team who's operating a Kubernetes or OpenShift cluster, use the following guides as references for implementating your deployment strategy:
  - [Kubernetes Deployment](https://docs.bentoml.org/en/latest/deployment/kubernetes.html)
  - [Knative Deployment](https://docs.bentoml.org/en/latest/deployment/knative.html)
  - [Kubeflow Deployment](https://docs.bentoml.org/en/latest/deployment/kubeflow.html)
  - [KFServing Deployment](https://docs.bentoml.org/en/latest/deployment/kfserving.html)
  - [Clipper.ai Deployment Guide](https://docs.bentoml.org/en/latest/deployment/clipper.html)



# Summary

This is what it looks like when using BentoML to serve and deploy a model in the cloud. BentoML also supports [many other Machine Learning frameworks](https://docs.bentoml.org/en/latest/examples.html) besides Scikit-learn. The [BentoML core concepts](https://docs.bentoml.org/en/latest/concepts.html) doc is recommended for anyone looking to get a deeper understanding of BentoML.

Join the [BentoML Slack](https://join.slack.com/t/bentoml/shared_invite/enQtNjcyMTY3MjE4NTgzLTU3ZDc1MWM5MzQxMWQxMzJiNTc1MTJmMzYzMTYwMjQ0OGEwNDFmZDkzYWQxNzgxYWNhNjAxZjk4MzI4OGY1Yjg) to follow the latest development updates and roadmap discussions.